In [ ]:
"""Tutorial: Second-Order Moller--Plesset Perturbation Theory (MP2)"""

__author__    = ["Justin M. Turney", "Marissa L. Estep"]
__credit__    = ["Justin M. Turney", "Marissa L. Estep"]

__copyright__ = "(c) 2014-2017, The Psi4NumPy Developers"
__license__   = "BSD-3-Clause"
__date__      = "2017-08-10"

# Integrals

In this tutorial you will learn how to compute the overlap and kinetic integrals used in electronic structure theory computations, as well as the dipole integrals used to compute the molecular dipole moment.  You will then incorporate your integral code into the restricted Hartree-Fock code you wrote in the first tutorial to test your integrals. Finally, using the converged one-particle density matrix from this restricted Hartree-Fock computation, you will compute the dipole moment of the molecule.

## Recurrence formula for one-electron integrals over Gaussian functions
### Cartesian Gaussian functions
The computation of molecular integrals can be considerably simplified by using recursion. We write the unnormalized Cartesian Gaussian function with origin at $\mathbf R$ as
\begin{equation}
\phi(\mathbf r; \alpha, \mathbf n, \mathbf R) = (x - \mathbf R_x)^{n_x} (y - \mathbf R_y)^{n_y} (z - \mathbf R_z)^{n_z} \exp[-\alpha (\mathbf r - \mathbf R)^2
\end{equation}
where $\mathbf r = (x, y, z)$ represents the coordinates of the electron, $\alpha$ is the orbital exponent, and $\mathbf n$ denotes a set of nonnegative integers $n_x$, $n_y$, and $n_z$:
\begin{equation}
\mathbf n = (n_x, n_y, n_z) \ .
\end{equation}
The normalization constant of the function is then given by
\begin{equation}
N(\alpha, \mathbf n) = \left(\frac{2\alpha}{\pi}\right)^{3/4} (4\alpha)^{(n_x + n_y + n_z)/2} [(2n_x-1)!!\ (2n_y-1)!!\ (2n_z-1)!!]^{-1/2}
\end{equation}

Letting $\lambda$ be defined as the sum of $n_x$, $n_y$, and $n_z$, one notes that it is closely related to the total angular momentum quantum number. $\lambda$ and $\mathbf n$ will hereafter be termed the angular momentum and the angular momentum index, respectively. The functions with $\lambda$ equal to $0,1,2,\ldots$, are referred to as $s,p,d,\ldots$, respectively. A set of $(\lambda+1)(\lambda+2)/2$ functions at $\mathbf R$ associated with the same angular momentum $\lambda$ and orbital exponent $\alpha$ constitute a *shell*, and the functions in the shell are *components* of the shell. The single component of the *s* shell with the angular momentum index $\mathbf 0 = (0,0,0)$ is usually designated *s*. The components of the *p* shell have the angular momentum indices $\mathbf 1_i\ (i=x,y,z)$, where  $\mathbf 1_i$ is defined by
\begin{equation}
\mathbf 1_i = (\delta_{ix},\delta_{iy},\delta_{iz})
\end{equation}
with Kronecker's deltas. These components will be designated $p_x$, $p_y$, and $p_z$. The components of the  *d* shell with the angular momentum indices $\mathbf 1_i + \mathbf 1_j\ (i,j=x,y,z)$ will be designated $d_{x^2}$, $d_{y^2}$, $d_{z^2}$, $d_{xy}$, $d_{xz}$, and $d_{yz}$. In later discussions all the components  in a shell are assumed to be exhausted as the basic functions, and to be treated together in the computation of molecular integrals.

First we give the basic equations for Cartesian Gaussian functions that will be utilized in the following derivations of molecular integrals. The Cartesian Gaussian functions satisfy the differential relation
\begin{equation}
\frac{\partial}{\partial R_i} \phi(\mathbf r; \alpha, \mathbf n, \mathbf R) = 2\alpha \phi(\mathbf r; \alpha, \mathbf n + \mathbf 1_i, \mathbf R) - N_i(\mathbf n)\phi(\mathbf r; \alpha, \mathbf n - \mathbf 1_i, \mathbf R) \qquad (i=x,y,z)
\end{equation}
where $N_i(\mathbf n)$, standing for $n_i$, is meant to take the value of the *i* component of the angular momentum index $\mathbf n$, and thus  $N_i(\mathbf 1_j)$ plays the same role as the Kronecker delta $\delta_{ij}$.

### Two-center overlap integrals
Two-center overlap integrals over unnormalized Cartesian Gaussian functions are of the form:
\begin{equation}
(\mathbf a|\mathbf b) = \int d\mathbf r\ \phi(\mathbf r; \alpha_a, \mathbf a, \mathbf A)\phi(\mathbf r; \alpha_b, \mathbf b, \mathbf B)
\end{equation}
According to the differential relation above the integral $(\mathbf a+\mathbf 1_i|\mathbf b)$ can be decomposed as
\begin{equation}
(\mathbf a+\mathbf 1_i|\mathbf b) = \frac{1}{2\alpha_a} \frac{\partial}{\partial \mathbf A_i} (\mathbf a|\mathbf b) + \frac{1}{2\alpha_a} N_i(\mathbf a) (\mathbf a - \mathbf 1_i|\mathbf b)
\end{equation}
Here the integral $(\mathbf a | \mathbf b)$ can be factored as
\begin{equation}
(\mathbf a | \mathbf b) = \kappa_{ab}\ I_x(n_{ax},n_{bx},n_{cx})\ I_y(n_{ay},n_{by},n_{cy})\ I_z(n_{az},n_{bz},n_{cz})
\end{equation}
where
\begin{align}
\kappa_{ab} =& \exp[-\xi (\mathbf A - \mathbf B)^2] \\
\xi =& \frac{\alpha_a \alpha_b}{\alpha_a + \alpha_b} \\
\alpha =& \alpha_a + \alpha_b \\
\mathbf P =& \frac{\alpha_a \mathbf A + \alpha_b \mathbf B}{\alpha_a + \alpha_b}
\end{align}
and
\begin{align}
I_i(n_{ai}, n_{bi}) =& \left(\frac{\pi}{\alpha_a + \alpha_b}\right)^{\tfrac{1}{2}} \\
 &\times \sum_{k_{ai}=0}^{n_{ai}} \sum_{k_{bi}=0}^{n_{bi}} \binom{n_{ai}}{k_{ai}} \binom{n_{bi}}{k_{bi}} \\
 &\times (\mathbf P_i - \mathbf A_i)^{n_{ai}-k_{ai}} (\mathbf P_i - \mathbf B_i)^{n_{bi}-k_{bi}} \\
 &\times \frac{(k_{ai} + k_{bi} - 1)!!}{(2\alpha)^{k_{ai}+k_{bi}}}
\end{align}
Differentiating $\kappa_{ab}$ and $I_i(n_{ai}, n_{bi})$ with respect to $A_i$, we have
\begin{equation}
\frac{1}{2\alpha_a}\ \frac{\partial}{\partial A_i} \kappa_{ab} = (P_i - A_i) \kappa_{ab}
\end{equation}
and
\begin{equation}
\frac{1}{2\alpha_a}\ \frac{\partial}{\partial A_i} I_i(n_{ai}, n_{bi}) = n_{ai}\left\{\frac{1}{2\alpha} - \frac{1}{2\alpha_a}\right\} I_i(n_{ai} - 1, n_{bi}) + n_{bi} \frac{1}{2\alpha} I_i(n_{ai}, n_{bi} - 1)
\end{equation}
Substituting these two equations into the expression for $(\mathbf a+\mathbf 1_i|\mathbf b)$ yields
\begin{equation}
(\mathbf a+\mathbf 1_i|\mathbf b) = (P_i - A_i)(\mathbf a|\mathbf b) + \frac{1}{2\alpha} N_i(\mathbf a)(\mathbf a-\mathbf 1_i|\mathbf b) + \frac{1}{2\alpha} N_i(\mathbf b)(\mathbf a|\mathbf b-\mathbf 1_i)
\end{equation}
and similarily
\begin{equation}
(\mathbf a|\mathbf b+\mathbf 1_i) = (P_i - B_i)(\mathbf a|\mathbf b) + \frac{1}{2\alpha} N_i(\mathbf a)(\mathbf a-\mathbf 1_i|\mathbf b) + \frac{1}{2\alpha} N_i(\mathbf b)(\mathbf a|\mathbf b-\mathbf 1_i)
\end{equation}
The integral over $s$ functions is given by
\begin{equation}
(\mathbf 0_A | \mathbf 0_B) = \left(\frac{\pi}{\alpha}\right)^{3/2} \exp[-\xi(\mathbf A-\mathbf B)^2]
\end{equation}

Given the recursive equations above, it should be seen that the recursion only depends on $\mathbf P-\mathbf A$, $\mathbf P-\mathbf B$, $\alpha$, and the target angular momentum of $\mathbf a$ and $\mathbf b$. The starting point of the recursion $(\mathbf 0_A | \mathbf 0_B)$ can be accounted for after the recursion has been performed.

### Recurrence expressions for overlap integrals over $s$, $p$, and $d$  Cartesian Gaussian funcions
\begin{align}
(s|s) =& \left(\frac{\pi}{\alpha}\right)^{3/2} \exp\left\{-\xi(\mathbf A-\mathbf B)^2\right\} \\
(p_i|s) =& (P_i - A_i)(s|s) \\
(p_i|p_j) =& (P_j - B_j)(p_i|s) + \frac{\delta_{ij}}{2\alpha}(s|s) \\
(d_{ij}|s) =& (P_j-A_j)(p_i|s) + \frac{\delta_{ij}}{2\alpha}(s|s) \\
(d_{ij}|p_k) =& (P_k-B_k)(d_{ij}|s) + \frac{\delta_{ik}}{2\alpha}(p_j|s) + \frac{\delta_{jk}}{2\alpha}(p_i|s) \\
(d_{ij}|d_{kl}) =& (P_l-B_l)(d_{ij}|p_k) + \frac{\delta_{il}}{2\alpha}(p_j|p_k) + \frac{\delta_{jl}}{2\alpha}(p_i|p_k) + \frac{\delta_{kl}}{2\alpha}(d_{ij}|s) \\
\\
& (i,j,k,l = x,y,z)
\end{align}

### Cartesian function ordering that Psi4 uses
In writing your recursion, you will need to be aware of how Psi4 orders Cartesian functions with higher angular momentum:
\begin{align}
\mathbf s &= s \\
\mathbf p &= p_x, p_y, p_z \\
\mathbf d &= d_{x^2}, d_{xy}, d_{xz}, d_{y^2}, d_{yz}, d_{z^2} \\
\mathbf f &= f_{x^3}, f_{x^2y}, f_{x^2z}, f_{xy^2}, f_{xyz}, f_{xz^2}, f_{y^3}, f_{y^2z}, f_{yz^2}, f_{z^3}
\end{align}

## One electron integrals
Now it is time to compute the following one-electron integrals:
- overlap
- kinetic
- dipole moment

Due to the added complexity of the nuclear attraction integrals and the two-electron repulsion integrals, we will not compute them in this tutorial.

### Kinetic energy integrals 
Before proceeding to an analysis of the kinetic energy integral, it will prove convenient to establish a short hard notation for integrals related to the overlap integral:
\begin{equation}
(0|0) = \int G(\alpha_1, \mathbf A, l_1, m_1, n_1) G(\alpha_2, \mathbf B, l_2, m_2, n_2)\ d\tau
\end{equation}
The symbol $<+\mathrm{n}|0>_{\mathrm{x}}$ will denote an integral of the form given by the above equation, except that the quantum number $l_1$ has been incremented by n. Similar notation applies to the $m_1$ and $n_1$ quantum numbers with the use of subscripts x and y.  We will also use the symbols $<0|+\mathrm{n}>_{\mathrm{x}}$, where we have incremented the quantum number $l_2$ by n. You should see the pattern.

The kinetic energy integral is defined as
\begin{equation}
T_{ij} = -\frac{1}{2} \int \phi_i(1) \nabla_1^2 \phi_j(1)\ d\tau_1
\end{equation}
and as
\begin{equation}
KE = - \frac{1}{2}  \int G(\alpha_1, \mathbf A, l_1, m_1, n_1) \left(\frac{\partial^2}{\partial x^2} + \frac{\partial^2}{\partial y^2} + \frac{\partial^2}{\partial z^2}\right) G(\alpha_2, \mathbf B, l_2, m_2, n_2)\ d\tau
\end{equation}

Evaluate the kinetic energy integrals for $I_x$, $I_y$, and $I_z$ and code them up to obtain $T_{ij}$. The equations that you obtain for these integrals are "unsymmetric" because the quantum numbers of the Gaussian-type function centered on $\mathbf B$ are altered while those of the Gaussian-type function on $\mathbf A$ are not.

Here is the symmetric form of the kinetic energy integral $I_x$ component.
\begin{equation}
I_x = \frac{1}{2}\Big\{l_1 l_2 (-1|-1)_{\mathrm{x}} + 4\alpha_1 \alpha_2 (+1|+1)_{\mathrm{x}} - 2\alpha_1 l_2 (+1|-1)_{\mathrm{x}} - 2\alpha_2 l_1 (-1|+1)_{\mathrm{x}}\Big\}
\end{equation}

### Dipole moment integrals
We discuss these non-energy integrals here because they are frequently used and closely connected with the overlap and kinetic energy integrals. The dipole moment is defined with respect to a point in space $\mathbf C$, which is almost always taken to be the center of mass. Fortunately for our purposes, Psi4 automatically moves the molecule to be centered on the center of mass and thus $\mathbf C$ is $\mathbf 0$. The dipole moment integral is written for the $x$ direction as
\begin{equation}
d_{\mathrm{x}} =  \int G(\alpha_1, \mathbf A, l_1, m_1, n_1) G(\alpha_2, \mathbf B, l_2, m_2, n_2) \mathrm{x_c}\ d\tau
\end{equation}
and similarly for the operators $\mathrm{y_c}$ and $\mathrm{z_c}$. A convenient procedure is to redefine $\mathrm{x_c}$ in terms of  $\mathrm{x_A}$ or $\mathrm{x_B}$, and we will use $\mathrm{x_A}$. Thus
\begin{equation}
\mathrm{x_c} = \mathrm{x} - \mathrm{C_x} = (\mathrm{x - A_x}) + (\mathrm{A_x - C_x}) = \mathrm{x_A + \mathbf {AC}_x}
\end{equation}
and we find
\begin{equation}
d_\mathrm{x} = (+1|0)_{\mathrm{x}} + \mathbf{AC}_{\mathrm{x}} (0|0)
\end{equation}
Similarily for $d_{\mathrm{y}}$ and $d_{\mathrm{z}}$.

#### The x component
\begin{align}
\mu_x =& - \mu_{\mathrm{elec}} + \mu_{\mathrm{nuclear}}\\
\mu_x =& - \sum_{\mu \nu}^{\mathrm{AO}} D_{\mu \nu} d_{\mu \nu}^{x} + \sum_A^N Z_A X_A
\end{align}

You can obtain the nuclear contribution to the dipole $\mu_{\mathrm{nuclear}}$ from Psi4 using

    psi4.core.nuclear_dipole(molecule).to_array()

#### The total dipole moment
\begin{equation}
\mu = \sqrt{\mu_x^2 + \mu_y^2 + \mu_z^2}
\end{equation}

## Load Psi4 and NumPy
As with the Hartree-Fock tutorial, we'll start by importing some useful modules and setting basic quantities for use in Psi4.

In [ ]:
import psi4
import numpy as np

# I'll show some plots
import matplotlib.pyplot as plt

# Load in the required options
import configparser
config = configparser.ConfigParser()
config.read('Options.ini')

molecule = psi4.geometry(config['DEFAULT']['molecule'])

# For your integrals code to match Psi4's ordering you must include "puream=0"
basis = psi4.core.BasisSet.build(molecule, 'BASIS', 'STO-3G', puream=0)
mints = psi4.core.MintsHelper(basis)

## Determine some information about the basis set

In [ ]:
# Look at the help documentation for the basis set
help(basis)
help(psi4.core.GaussianShell)

In [ ]:
print('The number of shells: %d' % (basis.nshell()))
print('The number of basis functions: %d' % (basis.nao()))

In [ ]:
# Print out some information about the shells in the basis set
for ishell in range(basis.nshell()):
    shell = basis.shell(ishell)
    print('Shell %d:' % (ishell))
    print('  Atom: %d' % (shell.ncenter))
    print('  AM: %d' % (shell.am))
    print('  # Cartesian functions: %d' % (shell.ncartesian))
    print('  # of primitive Gaussians: %d ' % (shell.nprimitive))
    print('  function_index: %d' % (shell.function_index))
    print('  center: %f, %f, %f' % (molecule.x(shell.ncenter), molecule.y(shell.ncenter), molecule.z(shell.ncenter)))

### Let's plot some of the functions to see what they look like

In [ ]:
# This is to show you have to access individual shells and access their coefficients and exponents.
shell0 = basis.shell(0)

# generate a list of points to evaluate the basis function (R=0 to 10 in 100 points)
R = np.linspace(0, 2, 100, endpoint=True)

# Plot each primitive
for idx in range(shell0.nprimitive):
    v = shell0.coef(idx) * np.exp(-shell0.exp(idx) * R**2)
    plt.plot(R, v)
plt.show()

### Write the recursion function
Write a function that performs the OS recursion for a given set of $\mathbf{PA}$, $\mathbf{PB}$, $\alpha$, and two angular momenta (for $\mathbf a$ and $\mathbf b$) values. At this point do not worry about solving $(\mathbf 0 | \mathbf 0)$ just set it to $1.0$ and you'll account for it later.  This function should also return three matrices containing the x, y, and z results.

In [ ]:
import numpy as np
from collections import namedtuple
RecursionResults = namedtuple('RecursionResults', ['x', 'y', 'z'])

def os_recursion(PA, PB, alpha, AMa, AMb):
    if len(PA) != 3 or len(PB) != 3:
        raise ""
        
    # Allocate the x, y, and z matrices.
    #  Why do I add 1 here?
    x = np.zeros((AMa+1, AMb+1))
    y = np.zeros((AMa+1, AMb+1))
    z = np.zeros((AMa+1, AMb+1))
    
    # Perform the recursion
    
    # Return the results
    return RecursionResults(x, y, z)

results = os_recursion([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], 1.0, 1, 1)
print(results)
print(results.x)

### Sample x, y, and z matrices

In [ ]:
import OS

In [ ]:
(x, y, z) = OS.os_recursion([1.0, 2.0, 3.0], [3.0, 2.0, 1.0], 1.0, 3, 3)
print('x=\n', x)
print('y=\n', y)
print('z=\n', z)

In [ ]:
dipole = [x.to_array() for x in mints.ao_dipole()]
print(dipole)
psi4.core.nuclear_dipole(molecule).to_array()

### Compute one-electron integrals
Now you should be ready to compute some integrals!

In [ ]:
import psi4
import numpy as np
from OS import os_recursion
# linalg package for matrix square root function:
from scipy import linalg as la
from collections import namedtuple
RecursionResults = namedtuple('RecursionResults', ['x', 'y', 'z'])

import configparser
config = configparser.ConfigParser()
config.read('Options.ini')

molecule = psi4.geometry(config['DEFAULT']['molecule'])
molecule.update_geometry()
basis = psi4.core.BasisSet.build(molecule, "BASIS", config['DEFAULT']['basis'], puream=0)
#This line is just for the two-electron integrals:
mints = psi4.core.MintsHelper(basis)

###################################
# Integral stuff
###################################
#Total number of AOs for molecule
naos = basis.nao()
#Matrix to store overlap integrals
S = np.zeros((naos, naos))
#Matrix to store kinetic energy integrals
K = np.zeros((naos, naos))
#Matrices to store dipole integrals
dipolex = np.zeros((naos, naos))
dipoley = np.zeros((naos, naos))
dipolez = np.zeros((naos, naos))

#Point with respect to which we
#define dipole moment
C = (0., 0., 0.)

for i in range(basis.nshell()): 
    for j in range(basis.nshell()):
        ishell = basis.shell(i)
        jshell = basis.shell(j)
        for p in range(ishell.nprimitive):
            for q in range(jshell.nprimitive):
                #p and q label primitives
                # l + m + n = angmom of shell
                AMa = ishell.am 
                AMb = jshell.am
                #
                Ax = molecule.x(ishell.ncenter)
                Ay = molecule.y(ishell.ncenter)
                Az = molecule.z(ishell.ncenter)
                Bx = molecule.x(jshell.ncenter)
                By = molecule.y(jshell.ncenter)
                Bz = molecule.z(jshell.ncenter)
                #alphas are exponents
                alphaa = ishell.exp(p)
                alphab = jshell.exp(q)
                #
                coefa = ishell.coef(p)
                coefb = jshell.coef(q)
                #
                alpha = alphaa + alphab
                Px = (alphaa*Ax + alphab*Bx)/(alpha)
                Py = (alphaa*Ay + alphab*By)/(alpha)
                Pz = (alphaa*Az + alphab*Bz)/(alpha)
                #
                PAx = Px - Ax
                PAy = Py - Ay
                PAz = Pz - Az
                # 
                PBx = Px - Bx
                PBy = Py - By
                PBz = Pz - Bz
                # 
                PA = (PAx, PAy, PAz)
                PB = (PBx, PBy, PBz)
                # 
                zeta = (alphaa*alphab)/(alpha)
                #
                kappa = (np.pi/alpha)**(3/2)*np.exp(-zeta*(Ax-Bx)**2)*np.exp(-zeta*(Ay-By)**2)*np.exp(-zeta*(Az-Bz)**2)
                #
                matx, maty, matz = os_recursion(PA, PB, alpha, AMa, AMb)
                #
                # Counter!!!
                counta = 0

                # l on atom one
                for k in range(AMa+1):
                    lo = AMa-k
                    # m on atom one
                    for l in range(AMa+1-lo):
                        mo = AMa - lo - l
                        # n on atom one
                        no = AMa - lo - mo
                        #
                        countb = 0
                        #print('AMa', lo, mo, no)
                        #
                        # l on atom two
                        for m in range(AMb+1):
                            lt = AMb-m
                            # m on atom two
                            for n in range(AMb+1-lt):
                                mt = AMb - lt - n
                                # n on atom two
                                nt = AMb - lt - mt
                                #print('AMb', lt, mt, nt)
                                # Set the matrix elements
                                S[ishell.function_index + counta, jshell.function_index + countb] += kappa*coefa*coefb*matx[lo,lt]*maty[mo,mt]*matz[no,nt]
                                #
                                # Kinetic energy integral stuff
                                Tx = (1/2)*(lo*lt*matx[lo-1,lt-1] + 4*alphaa*alphab*matx[lo+1,lt+1] - 2*alphaa*lt*matx[lo+1,lt-1] - 2*alphab*lo*matx[lo-1,lt+1])
                                Ty = (1/2)*(mo*mt*maty[mo-1,mt-1] + 4*alphaa*alphab*maty[mo+1,mt+1] - 2*alphaa*mt*maty[mo+1,mt-1] - 2*alphab*mo*maty[mo-1,mt+1])
                                Tz = (1/2)*(no*nt*matz[no-1,nt-1] + 4*alphaa*alphab*matz[no+1,nt+1] - 2*alphaa*nt*matz[no+1,nt-1] - 2*alphab*no*matz[no-1,nt+1])
                                #
                                K[ishell.function_index + counta, jshell.function_index + countb] += kappa*coefa*coefb*Tx*maty[mo,mt]*matz[no,nt] 
                                K[ishell.function_index + counta, jshell.function_index + countb] += kappa*coefa*coefb*matx[lo,lt]*Ty*matz[no,nt] 
                                K[ishell.function_index + counta, jshell.function_index + countb] += kappa*coefa*coefb*matx[lo,lt]*maty[mo,mt]*Tz
                                # 
                                dipolex[ishell.function_index + counta, jshell.function_index + countb] += kappa*coefa*coefb*(matx[lo+1,lt] + (Ax)*matx[lo,lt])*maty[mo,mt]*matz[no,nt]
                                dipoley[ishell.function_index + counta, jshell.function_index + countb] += kappa*coefa*coefb*matx[lo,lt]*(maty[mo+1,mt] + (Ay)*maty[mo,mt])*matz[no,nt]
                                dipolez[ishell.function_index + counta, jshell.function_index + countb] += kappa*coefa*coefb*matx[lo,lt]*maty[mo,mt]*(matz[no+1,nt] + (Az)*matz[no,nt])
                                #
                                countb = countb + 1
                        #
                        counta = counta + 1


###################################
# UHF
###################################
# Number of alpha orbitals
nalph = int(config['DEFAULT']['nalpha'])
# Number of beta orbitals
nbeta = int(config['DEFAULT']['nbeta'])

# Maximum number of SCF iterations
SCF_MAX_ITER = int(config['SCF']['max_iter'])

# Overlap integrals
S = S
# Kinetic integrals
T = K
# Potential integrals
V = mints.ao_potential().to_array()
# Two-electron repulsion integrals
I = mints.ao_eri().to_array()

# Guess Hamiltonian
H = T + V

#Square root of overlap matrix
#to transform Fock matrix
A = la.sqrtm(la.inv(S))

# Guess Fock operators
# using one-electron Hamiltonian
Ftalph = A.dot(H).dot(A)
Ftbeta = A.dot(H).dot(A)
#
ealph, Calph = np.linalg.eigh(Ftalph)
ebeta, Cbeta = np.linalg.eigh(Ftbeta)
Calph = A.dot(Calph)
Cbeta = A.dot(Cbeta)
# UHF alpha and beta C matrices
Calph = Calph[:, :nalph]
Cbeta = Cbeta[:, :nbeta]

# UHF alpha and beta initial density matrices
Dalph = np.einsum('pi,qi->pq', Calph, Calph)
#print(Dalph)
Dbeta = np.einsum('pi,qi->pq', Cbeta, Cbeta)
#print(Dbeta)

# SCF iterations
EUHF = 0.0
EoldUHF = 0.0
Doldalph = np.zeros_like(Dalph)
Doldbeta = np.zeros_like(Dbeta)

for iteration in range(1, SCF_MAX_ITER+1):

    #Make alpha and beta Fock matrix components
    Jalph = np.einsum('pqrs,rs->pq', I, Dalph)
    Jbeta = np.einsum('pqrs,rs->pq', I, Dbeta)
    Kalph = np.einsum('prqs,rs->pq', I, Dalph)
    Kbeta = np.einsum('prqs,rs->pq', I, Dbeta)

    #Extra energy term
    alphbeta = np.einsum('pq,pq->', Jalph, Dbeta)

    #Make alpha and beta Fock matrices
    Falph = H + Jalph + Jbeta - Kalph
    Fbeta = H + Jalph + Jbeta - Kbeta

    twoelec = np.einsum('pq,pq->', Falph, Dalph) + np.einsum('pq,pq->', Fbeta, Dbeta)

    #Contract one-electron terms
    Halph = np.einsum('pq,pq->', H, Dalph)
    Hbeta = np.einsum('pq,pq->', H, Dbeta)
    oneelec = Halph + Hbeta

    # Calculate UHF energy
    #EUHF = oneelec + twoelec + molecule.nuclear_repulsion_energy()
    EUHF = 0.5*(oneelec + twoelec) + molecule.nuclear_repulsion_energy()
    print('UHF iteration %3d: energy %20.14f dE %1.5E' % (iteration, EUHF, (EUHF - EoldUHF)))

    if (abs(EUHF - EoldUHF) < 1.e-10):
        break
    EoldUHF = EUHF
    #Store old density matrices for alpha and beta electrons
    Doldalph = Dalph
    Doldbeta = Dbeta

    # Transform the alpha and beta Fock matrices
    Ftalph = A.dot(Falph).dot(A)
    Ftbeta = A.dot(Fbeta).dot(A)

    # Diagonalize the alpha and beta Fock matrices
    ealph, Calph = np.linalg.eigh(Ftalph)
    ebeta, Cbeta = np.linalg.eigh(Ftbeta)

    # Construct new UHF eigenvector matrices
    Calph = A.dot(Calph)
    Cbeta = A.dot(Cbeta)

    # Form the alpha and beta density matrices
    Calph = Calph[:, :nalph]
    Dalph = np.einsum('pi,qi->pq', Calph, Calph)
    Cbeta = Cbeta[:, :nbeta]
    Dbeta = np.einsum('pi,qi->pq', Cbeta, Cbeta)

print('Final UHF energy: %20.14f' % (EUHF))

Density = Dalph + Dbeta

###################################
# Dipole Moment
###################################

dipolenuclear = psi4.core.nuclear_dipole(molecule).to_array()
dipolemux = - (np.einsum('pq,pq->', Density, dipolex)) + dipolenuclear[0]
dipolemuy = - (np.einsum('pq,pq->', Density, dipoley)) + dipolenuclear[1]
dipolemuz = - (np.einsum('pq,pq->', Density, dipolez)) + dipolenuclear[2]

dipolemu = np.sqrt(dipolemux**2 + dipolemuy**2 + dipolemuz**2)
print(dipolemu)

#### Compare to Psi4
Check your energy and dipole moment against the energy and dipole moment Psi4 outputs: